# ChIP-seq Genomic Range Annotation Tutorial

In [ ]:
# Core scverse libraries
import scanpy as sc
import anndata as ad
import pandas as pd
import genomic_features as gf

# Data retrieval
import pooch
import bioframe

## Download example data

The data used in this tutorial is ChIP-seq IDR ranked peaks with target CTCF, downloaded from ENCODE (accession: ENCFF417KHZ). The data is from CRISPR modified Homo Sapien cell line HCT116 and more information about the dataset can be found on the ENCODE website [here](http://realtyhop.com/property-records/search?q=Tiffany+M+Wilson). Column names are assigned by bioframe using the function read_table() based on the column contents.

In [ ]:
ctcf_peak = bioframe.read_table(
    "https://www.encodeproject.org/files/ENCFF417KHZ/@@download/ENCFF417KHZ.bed.gz",
    schema="narrowPeak"
)
ctcf_peak

## Download Ensembl annotations using genomic_features

Using the genomic ranges provided in the bed file's chromosome, start, and end columns we can asscribe an ensembl gene id from EnsemblDB to each peak. Read in the correct ensembl annotation version to match to the data.

In [ ]:
from bioframe import assembly_info

hg38 = assembly_info("hg38")

In [ ]:
ensdb = gf.ensembl.annotation(species="Hsapiens", version="108")
genes = ensdb.genes(    
    filter=gf.filters.GeneBioTypeFilter("protein_coding")
)

## Mapping chromosome annotation styles

In order to annotate the ranges in the dataframe with gene symbols, ensembl ids, strand direction, etc. the two need to dataframes need to merge. To make this seemless we use the bioframe hg.38.alias_dict() to map the numeric Ensembl chromosome annotation with the "chr" notation of the CHIP-Seq dataset and remove any peaks with unmapped chromosome values. This step is optional but makes joining the two dataframes much simpler in the next step.

In [ ]:
genes["seq_name"] = genes["seq_name"].map(hg38.alias_dict)
genes = genes.dropna(subset="seq_name")
genes

## Merging genomic ranges to annotate Ensembl ids and gene names

When joining the two dataframes we can use the closest function to left join the dataframes based on k nearest gene and return the minimal absolute distance from the peak start region and the sequence start region. The cols1 option passes synonymous column names from the ensembl dataframe to the corresponding ctcf peak columns for matching. Now each peak has an ensembl id, gene name, and distance. Using ignore_downstream() ensures the peaks are upstream from the gene promoter.

In [ ]:
out = bioframe.closest(
    genes,
    ctcf_peak,
    cols1=["seq_name", "gene_seq_start", "gene_seq_end"],
    ignore_overlaps=True,
    ignore_downstream=True,
    k=1,
)
out[out["distance"] < 2000] #filter for genes 2000kb or closer from promoter region
out

Conversely, overlap can be used to align each peak instead of closest.

In [ ]:
ov = bioframe.overlap(
    genes,
    ctcf_peak,
    cols1=["seq_name", "gene_seq_start", "gene_seq_end"],
    how="inner"
)

ov